In [ ]:
"""
make sure you have the necessary packages installed to use the cstdn module

Directory structure should look something like this:
    configurations
        config_1
            vol_1
            vol_2
        config_2
            vol_1
            vol_2

I think the only name that matters is the name of the volume, so you can have:
    structures
        str_1
            vol_1
            vol_2
        str_2
            vol_1
            vol_2
But, I'm not 100% sure. At any rate the tutorial will be easier to follow with
the config name scheme

"""

In [1]:
"""
add the path of your version of vasp-job-automation to the system path
"""

sys.path.append('/storage/home/lam7027/bin/vasp-job-automation') #Change this to your path
import cstdn
import sys
import os
import pandas as pd
import yaml



#this took me 10s to import. a testement that I need to move stuff out of cstdn.py

In [ ]:
"""
To extract data from a single config and then plot that data,
"""

In [ ]:
# change the list to the ions you want to plot the magnetic moment data for
ion_list = [i for i in range(1, 9)] # recall that range(n, m) = [n, ..., m-1]

In [ ]:
"""
We can extract the data to a dataframe using this function

Change path to the 'config_x' dir. this is the directory that cointains
all the vol_x dirs which contain the OUTCAR and OSZICAR files for that volume

If the OUTCAR and OSZICAR files have different names, specify them,
    ex: outcar_name='OUTCAR.3static'
"""

df = cstdn.extract_config_data('/path/to/config_x', ion_list, outcar_name='OUTCAR.2relax', oszicar_name='OSZICAR.2relax') 

In [ ]:
"""
now that we have the data for that config, we can plot it.
"""
fig1 = cstdn.plot_mv(df, show_fig=True)
fig2 = cstdn.plot_ev(df, show_fig=True) 


In [ ]:
"""
mBM4 is the default eos fitting method for plot_ev().
    If you want to change it, some options include:
        plot_ev(df, eos_fitting='mBM5', show_fig=True)
        plot_ev(df, eos_fitting='vinet', show_fig=True)
        plot_ev(df, eos_fitting='all', show_fig=True)
        plot_ev(df, eos_fitting=None, show_fig=True)
"""

fig3 = cstdn.plot_ev(df, eos_fitting='all', show_fig=True)

# the plots still need some work to look nice, but they are functional

In [ ]:
"""
If we want to extract data from multiple configurations we can use a loop to iterate
over the config dirs.
"""

In [ ]:
"""
Change this to the directory that contains all the 'config_x' dirs
"""

configurations_dir = 'your/path/to/configurations'


In [ ]:
"""
Don't change this. This is a list of all the 'config_x' dirs in the configurations_dir
"""

config_dirs = [os.path.join(configurations_dir, d) for d in os.listdir(configurations_dir) if os.path.isdir(os.path.join(configurations_dir, d))]


In [ ]:
"""
The following code
    iterates over the config dirs and extracts the data from each config
    makes a list of the dataframes for each config
    concatenates the dataframes into a single dataframe that contains all the data.

Optionally, the commented out code allows you to make individual plots and/or save
them to a png file for each config as you iterate over them.
"""

"""
Change the outcar_name and oszicar_name if necessary. ex: outcar_name='OUTCAR.3static'
"""
df_list = []
for config_dir in config_dirs:
    config_df = cstdn.extract_config_data(config_dir, ion_list, outcar_name='OUTCAR', oszicar_name='OSZICAR')
    # ev_fig = cstdn.plot_ev(config_df, None, show_fig=False) # show_fig = True will open up a browser window with the plot
    # mv_fig = cstdn.plot_mv(config_df, show_fig=False) # show_fig = True will open up a browser window with the plot
    # ev_fig.write_image(f'{os.path.basename(config_dir)}_ev_fig.png')
    # mv_fig.write_image(f'{os.path.basename(config_dir)}_mv_fig.png')
    df_list.append(config_df)
df = pd.concat(df_list, ignore_index=True)

In [ ]:
"""
To plot all the EV curves together, we simply call the plot_ev function as before
"""

multiple_ev_fig = cstdn.plot_ev(df, eos_fitting='mBM4', show_fig=True)


In [ ]:
# save the pandas dataframe to a json file
df.to_json('data.json')
    #json makes more sense for tabular data than yaml,
    #but not sure how Dr. Shang feels on the matlab end.

# make a dictionary from the dataframe
data_dict = df.to_dict()

# save a yaml file from the dictionary
with open('data.yaml', 'w') as f:
    yaml.dump(data_dict, f)


In [ ]:
"""
The funciton used above, extract_config_data(), gives magnetic data only
for the last ionic step of the OUTCAR file.

There is, however, a function that extracts the magnetic data for all ionic steps.
change the outcar_path
"""

all_magnetic_data = cstdn.extract_mag_data(outcar_path='path/to/OUTCAR')
print(all_magnetic_data)